In [ ]:
#%matplotlib widget
from matplotlib import pyplot as plt
import numpy as np
import scipy.io as scio
import netCDF4
import sys
sys.path.append('./modules/')
from emit_tools import emit_xarray
import xarray as xr
import rioxarray
import holoviews as hv
import hvplot.xarray
import rasterio
from rasterio.plot import show
from osgeo import osr, gdal

### Read in EMIT data

In [ ]:
fp = '../data/EMIT_L2A_RFL_001_20230728T214106_2320914_002.nc'
ds_geo = emit_xarray(fp, ortho=True)
ds_geo

In [ ]:
ds_geo.sel(wavelengths=850, method='nearest').hvplot.image(cmap='viridis', frame_width=500, geo=True, tiles='EsriImagery').opts(
    xlabel=f'{ds_geo.longitude.long_name} ({ds_geo.longitude.units})', ylabel=f'{ds_geo.latitude.long_name} ({ds_geo.latitude.units})')

### ECOSTRESS

In [ ]:
geotiff_path = '../data/ECOv002_L2T_LSTE_28691_004_11SLT_20230728T214058_0710_01_LST.tif'
geotiff_da = rioxarray.open_rasterio(geotiff_path)

In [ ]:
geotiff_ds = geotiff_da.to_dataset('band')
# Rename the variable to a more useful name
geotiff_ds = geotiff_ds.rename({1: 'LSTE'})

In [ ]:
geotiff_ds

In [ ]:
geotiff_ds.LSTE.plot()

### Interpolate ECOSTRESS to EMIT data

In [ ]:
# ecostress_resamp = geotiff_ds.interp_like(ds_geo)

# *** Under construction *** #

### Other ways to read in EMIT data

In [ ]:
#emit_data = scio.netcdf.netcdf_file('../data/EMIT_L2A_RFL_001_20230728T214106_2320914_002.nc',mode='r')
emit_data = netCDF4.Dataset('../data/EMIT_L2A_RFL_001_20230728T214106_2320914_002.nc')
emit_data_uncert = netCDF4.Dataset('../data/EMIT_L2A_RFLUNCERT_001_20230728T214106_2320914_002.nc')
emit_data_mask = netCDF4.Dataset('../data/EMIT_L2A_MASK_001_20230728T214106_2320914_002.nc')

In [ ]:
plt.figure()
plt.plot(emit_data['/sensor_band_parameters/wavelengths/'][:])
plt.plot(emit_data.groups['sensor_band_parameters'].variables['wavelengths'][:])

In [ ]:
emit_data.groups['location'].variables.keys()

In [ ]:
emit_data.variables['reflectance'].shape

In [ ]:
emit_data.groups['location'].variables['lon'].shape

In [ ]:
emit_data.groups['location'].variables['glt_x'].shape

In [ ]:
plt.figure()
plt.imshow(emit_data.groups['location'].variables['lon'])

In [ ]:
emit_data_mask['mask'].shape

In [ ]:
plt.figure()
plt.imshow(emit_data_mask['mask'][:,:,7])
plt.colorbar()

### Other ways to read in ECOSTRESS data

In [ ]:
ds = gdal.Open('../data/ECOv002_L2T_LSTE_28691_004_11SLT_20230728T214058_0710_01_LST.tif')

In [ ]:
img = rasterio.open('../data/ECOv002_L2T_LSTE_28691_004_11SLT_20230728T214058_0710_01_LST.tif')
show(img)

In [ ]:
img